In [1]:
!pip install pytz

In [3]:
# 引用boto3套件
import boto3
# 引用botocore.exceptions套件中ClientError的方法
from botocore.exceptions import ClientError
# 引用json套件
import json
# 引用datetime套件
import datetime
# 引用pytz套件(可取得local time,utc time)
import pytz

In [4]:
# 先設定epurl變數為sqs container連線位置，此變數後續可直接在endpoint_url使用
epurl="http://cc104.sqs.local:9324"
#============================================
# 先取得local time,utc time
tz = pytz.timezone('Asia/Taipei')
utc = pytz.utc
i=1
#============================================
#取得local time,utc time
#print(pytz.country_timezones('tw'))
tz = pytz.timezone('Asia/Taipei')
utc = pytz.utc
#============================================

In [6]:
print("="*40)
# 用boto3的session方法調用container中.aws/目錄下的環境變數設定檔，並存入變數user_session中
user_session=boto3.session.Session(profile_name='user1')
# 用client方法連線sqs container
client = user_session.client('sqs', endpoint_url=epurl)
#連線成功，輸出Local time、UTC time相關訊息
print("Local time:",datetime.datetime.now(tz))
print("  UTC time: ",datetime.datetime.now(pytz.utc),"\n",client,"\n~~連線成功~~")
print("="*40)

Local time: 2019-02-10 15:19:13.157024+08:00
  UTC time:  2019-02-10 07:19:13.159477+00:00 
~~連線成功~~


In [24]:
# 用create_queue方法創建名為test2的隊列
queue = client.create_queue(QueueName='test2', Attributes={'DelaySeconds': '5'})
#輸出回應內容
print("="*40)
print("用create_queue方法創建標準queue(QueueName為test2)\nQueueUrl： ",queue['QueueUrl'],"\nHTTPStatusCode：",queue['ResponseMetadata']['HTTPStatusCode'])
print("="*40)

用create_queue方法創建標準queue(QueueName為test2)
QueueUrl：  http://cc104.sqs.local:9324/queue/test2 
HTTPStatusCode： 200


In [25]:
#讀取test2隊列
client2 = user_session.resource('sqs', endpoint_url=epurl)
#輸出回應內容
queue1 = client2.get_queue_by_name(QueueName='test2')
print("="*40)
print("讀取queue(QueueName為test2)\nQueue2Url：",queue1.url)
response = client.receive_message(QueueUrl=queue1.url)
print("ResponseMetadata:")
print(response['ResponseMetadata'])
print("="*40)

讀取queue(QueueName為test2)
Queue2Url： http://cc104.sqs.local:9324/queue/test2
ResponseMetadata:
{'RequestId': '00000000-0000-0000-0000-000000000000', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'akka-http/10.1.3', 'date': 'Sun, 10 Feb 2019 07:25:59 GMT', 'content-type': 'text/plain; charset=UTF-8', 'content-length': '753'}, 'RetryAttempts': 0}


In [125]:
#用send_message方法將message傳送到test2 queue中
print("="*40)
#設定單個send Message1的相關屬性並傳送
response_s = queue1.send_message(
    MessageBody='hahaha~~~Hello',
    MessageAttributes={"msn":{"DataType":"String","StringValue":"communication1"}},
)
print("傳送 message1至test2隊列...")
print("MessageId:",response_s['MessageId'])
print("MD5OfMessageBody:",response_s.get('MD5OfMessageBody'),'\n')
print("="*40)
#設定單個send Message2的相關屬性並傳送
response_s = queue1.send_message(
    MessageBody='Hello~~~~wahahaha',
    MessageAttributes={"msn":{"DataType":"String","StringValue":"communication2"}},
)
print("傳送 message2至test2隊列...")
print("MessageId:",response_s.get('MessageId'))
print("MD5OfMessageBody:",response_s.get('MD5OfMessageBody'),'\n')
print("="*40)

傳送 message1至test2隊列...
MessageId: 777f2faf-a9b6-4497-b92a-2b2db5a9e9eb
MD5OfMessageBody: 56f50a1fb6f14992da7d3dc51d437521 

傳送 message2至test2隊列...
MessageId: 6224b7b9-e822-4a99-8a0a-f3fb2b1bc712
MD5OfMessageBody: c4ae2c026057b20d3d17998d4cb248a0 



In [126]:
#將test2隊列中的message取出(一次只能rolling出一個`)
for kk in queue1.receive_messages():
    print("="*40)
    try:
        print('隊列:\t',kk.queue_url)
        print('訊息 ID:\t',kk.message_id)
        print('訊息 Body:\t',kk.body)
        print('訊息 md5 Body:\t',kk.md5_of_body)
        #kk.delete()
    except:
        print(".....")
    print("="*40)

隊列:	 http://cc104.sqs.local:9324/queue/test2
訊息 ID:	 8e0723d9-5e2e-482b-8a9f-ffcaabcfe3ce
訊息 Body:	 hahaha~~~Hello
訊息 md5 Body:	 56f50a1fb6f14992da7d3dc51d437521


In [127]:
#將test隊列中的message取出
for d in queue1.receive_messages(MessageAttributeNames=['msn']):
    if d.message_attributes is not None:
        ma = d.message_attributes.get('msn').get('StringValue')
        mid = d.message_attributes.get('msn')
        print(ma,mid)
        #d.delete()
        print("已刪除AttributeNames為msn且內容為",ma,"訊息")

communication2 {'StringValue': 'communication2', 'DataType': 'String'}
已刪除AttributeNames為msn且內容為 communication2 訊息


In [128]:
i=1
#用for輸出所有queue
print("================================\n")
for j in client2.queues.all():
    print("第",i,"個 queue ： ", j.url)
    i+=1
print("================================\n")


第 1 個 queue ：  http://cc104.sqs.local:9324/queue/test2
第 2 個 queue ：  http://cc104.sqs.local:9324/queue/queue1
第 3 個 queue ：  http://cc104.sqs.local:9324/queue/test1
第 4 個 queue ：  http://cc104.sqs.local:9324/queue/test3

